In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://www.deccanherald.com/sites/dh/files/styles/article_detail/public/article_images/2020/05/19/netflix-1292029648-1585846485.jpg?itok=X7hobtZ6)

## FirstSection: Durations/Categories for Films
Firstly, I uploaded data file to my notebook for examination. After this I used some methods from pandas to detect what kind of data I would deal with.

In [ ]:
data = pd.read_csv("../input/netflix-shows/netflix_titles.csv") #pathfile


In [ ]:
data.head()

In [ ]:
print(data.columns)
print(data.info())

I saw there are 12 columns in this data set, then I thought I can use scatter plot from matplotlib to see correlation between "listed_in" class(categories) and durations for especially films. But firstly I needed to create new integer column as they were just basic strs.


# Listed_İn Parts

In [ ]:
dataFilms = data[data.type == "Movie"]
dataFilms.head()

Firstly, I eliminated TV shows, because I thought that it is better to examine both of them seperately at first.

In [ ]:
dataFilms.info()


Secondly, I need to determine missing values, because it is really important for especially listed in and durations.

In [ ]:
dataFilms.isna().sum()


I noticed that there is no missing values in duration and listed_in parts, so I am glad to continue to my work by determining what kind of listed_in variables I have got. I know it is a really mess and combination of many  other variables but I needed to determine them firstly.  

In [ ]:
dataFilms["listed_in"].unique

I thought that this ", " is really helpful for my project to seperate species in more effective way. So firstly I needed to try it.

In [ ]:
#For instance we can examine sixth line.

a = "International Movies, Sci-Fi & Fantasy, Thrillers"
b = a.split(", ")
print(b)



I should use here "for loop" of course so:

In [ ]:
listedin = []
for each in dataFilms["listed_in"]:
    each = each.split(", ")
    listedin = listedin + each
    
listedin = list(dict.fromkeys(listedin))
print(listedin)    

Finally, I could reach 19 main listed_in's species. After this I need to focus on duration parts.

# Duration Parts

In [ ]:
dataFilms["duration"].head()


So I could see that duration parts are written in strings, so ı need to create new dataframe by transforming them integer.

In [ ]:
#for example we can use here apply method

def splitandTransform(x):
    x = x.split(" ")
    try:
        x = int(x[0])
        return x
    except ValueError:
        pass

#I should try this firstly so:


In [ ]:
dataFilms["durationINT"] = dataFilms.duration.apply(splitandTransform)
dataFilms["durationINT"].head() #I added new column because I did not want to change original data.

So anymore, we can see duration-indexing line plot for films. 

# First Graph

In [ ]:
dataFilms.durationINT.plot(kind ='line', color = 'green',label="Duration",linewidth=1,alpha = 0.5, grid = True,)
plt.legend()
plt.xlabel("Index")
plt.ylabel("Duration:Minutes")
plt.title("Duration of Line Plots")
fig =plt.gcf()
fig.set_size_inches(18.5, 10.5)

plt.show()

But I want them to catogorize with their listedin profiles so I need to scan for every film for each category and I need to define their avarage durations. But I know that one film can be placed in different categories but I do not seem this is a problem for me.

In [ ]:
"""
This section was really complex for me and it takes time to calculate, 
as I am beginner I am sorry for this, I hope that I will solve this in more effective way.
"""
mlist = []
for a in listedin:
    tlist = []
    for b in dataFilms["show_id"]:
        if a in str(dataFilms[dataFilms['show_id'] == b]['listed_in']):
            tlist = tlist + list(dataFilms[dataFilms['show_id'] == b]['durationINT'])
        else:
            pass
    result = int(sum(tlist)/len(tlist))
    mlist.append(result)
print(mlist)            
    
    
            
            

After calculating average of durations each listed-in I was ready to graph it :)

In [ ]:
dataFilms2 = dict(zip(listedin,mlist)) 
dataFilms2 = pd.Series(dataFilms2)
print(dataFilms2)



I have created series instead of DataFrame, because Pandas requires it, but as receiving lot of indexing errors from Panda, ı decided to transform this into dataframe after this. I do not know why I could not create a dataframe at first, if you know you can answer this :)

In [ ]:
df = pd.DataFrame({'Listed_Species':dataFilms2.index, 'Avarage_Durations':dataFilms2.values})
df

After this, I need to do some balance for visualization to Listed_Species

In [ ]:
c = "Movies"
def delMovies(x):
   
    if c in str(x).split(" "):
        a = str(x).replace("Movies","")
        if a == "":
            return x
        else:
            return a
    else:
        return x
        

df["Listed_Species"] = df.Listed_Species.apply(delMovies)
df.iloc[17,0] = "General Movies"
print(df)

In [ ]:
df_A = df.iloc[:10,:10]
df_A.plot(kind = "scatter", x = "Listed_Species", y = "Avarage_Durations", alpha = 1, color ="red", grid = True, s = 500)
plt.xlabel("Listed_Species")
plt.ylabel("Avarage_Durations")
plt.title("Durations According To Species 1")
fig =plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

df["Avarage_Durations"]=df["Avarage_Durations"].astype(float)
df_B = df.loc[::-1,::-1]
df_B = df_B.iloc[:10,:10]
df_B.plot(kind = "scatter", x = "Listed_Species", y = "Avarage_Durations", alpha = 1, color ="blue", grid = True, s = 500)
plt.xlabel("Listed_Species")
plt.ylabel("Avarage_Durations")
plt.title("Durations According To Species 2")
fig =plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.show()

It seems like classic and musicals shares the highest durations. As we can see that especially stand-up comedies with their one hour rate(aproximately) have the lowets durations. Another one children & family, maybe this can be related about the real duration of focusing capability of youngsters. This Data Analyze  was my first one. After this section, we will continue with another elements of this data; for instance countries with durations, release dates with species. Of course, we can apply same structure to TV series with the numbers of season. 

Hopefully you enjoy that. Thank you for your reading. I just started to code two week ago, so I am sure that there are more effective ways to do this. I believe that I will learn them.

